In [1]:
import pandas as pd
import numpy as np

from BertClassifier import BertClassifier
from config import BASE_DIR
text_files_dir = f"{BASE_DIR}/data/hebrew_processed_files/2024-04-03_sectarian_classification"

In [2]:


text = pd.read_csv(f"{text_files_dir}/DSS_sectarian_nonbib_text.txt", header=None, names=["text"])
labels = pd.read_csv(f"{text_files_dir}/DSS_sectarian_nonbib_labels.txt",sep="\t",header=None, names=["place", "training", "label"])
assert text.shape[0]==labels.shape[0]

In [3]:
df = pd.concat([text,labels],axis=1)[["text","training","label"]]
df = df.dropna()
df.groupby("label").count()

,text,training
label,,
non_sectarian_texts,246,246
sectarian_texts,1730,1730


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Removing rows with missing 'text' values
df_clean = df.dropna(subset=['text'])

# Vectorizing the text column
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df_clean['text'])

# Preparing the labels
y = df_clean['label']

# Splitting the dataset into training and test sets based on the 'training' column
X_train = X[df_clean['training'] == 'train']
y_train = y[df_clean['training'] == 'train']
X_test = X[df_clean['training'] == 'test']
y_test = y[df_clean['training'] == 'test']

# Training a logistic regression model
model = LogisticRegression(max_iter=1000)  # Increased max_iter for convergence
model.fit(X_train, y_train)

# Predicting and evaluating the model
y_pred = model.predict(X_test)
classification_report_result = classification_report(y_test, y_pred)

classification_report_result


/Users/yonatanlou/.pyenv/versions/3.10.0/envs/QumranNLP/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yonatanlou/.pyenv/versions/3.10.0/envs/QumranNLP/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yonatanlou/.pyenv/versions/3.10.0/envs/QumranNLP/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

'                     precision    recall  f1-score   support\n\nnon_sectarian_texts       0.00      0.00      0.00        53\n    sectarian_texts       0.87      1.00      0.93       370\n\n           accuracy                           0.87       423\n          macro avg       0.44      0.50      0.47       423\n       weighted avg       0.77      0.87      0.82       423\n'

In [5]:
print(classification_report_result)

                     precision    recall  f1-score   support

non_sectarian_texts       0.00      0.00      0.00        53
    sectarian_texts       0.87      1.00      0.93       370

           accuracy                           0.87       423
          macro avg       0.44      0.50      0.47       423
       weighted avg       0.77      0.87      0.82       423


In [6]:
from transformers import BertTokenizer
from BertClassifier import BertClassifier
EPOCHS = 5
model = BertClassifier()
tokenizer = BertTokenizer.from_pretrained("onlplab/alephbert-base")
LR = 1e-6
# section_type = ["nonbib", "bib"]
section_type = ['sectarian_texts', 'non_sectarian_texts']


train_data, train_label = np.array([]), np.array([], dtype=int)
val_data, val_label = np.array([]), np.array([], dtype=int)
test_data, test_label = np.array([]), np.array([], dtype=int)

# size = [0 for i in section_type]
# for i in range(len(section_type)):
#     all_data = np.array([])
#     all_labels = np.array([], dtype=int)
#     section = section_type[i]
#     book_dict, book_to_section = generate_books_dict(
#         BOOKS_TO_RUN_ON, "all_sectarian_texts.yaml"
#     )
#     data = parser_data.get_dss_data(book_dict, section)
#     for book_name, book_data in data.items():
#         if len(book_data) < 100:
#             print(book_name)
#             continue
#         book_scores = [section, book_name]
#         samples, sample_names = parser_data.get_samples(book_data)
#         preprocessed_samples = aleph_bert_preprocessing(samples)
#         labels = [i for _ in range(len(samples))]
#         all_data = np.concatenate((all_data, preprocessed_samples))
#         all_labels = np.concatenate((all_labels, labels))
#     size[i] = len(all_data)

Some weights of BertModel were not initialized from the model checkpoint at onlplab/alephbert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
df

,text,training,label
0,שבועות. ובארבעה לשבוע החמישי שמחו וידע אדם שני...,train,non_sectarian_texts
1,תחת הבל כיא הרגו קין. בשבוע הששי הוליד את אזור...,train,non_sectarian_texts
2,ובשנת אחת לשבוע הריאשון ליובל החמישי נבנו הבתי...,train,non_sectarian_texts
3,עוד תשעה בנים ובשבוע החמישי. לקח אנוש את אחותו...,train,non_sectarian_texts
4,לקח לו קינן אשה את אחותו מהללת לאשה. ותלד לו ב...,train,non_sectarian_texts
...,...,...,...
1973,מעיר הקדש וישענו על אל בקץ מעל ישראל ויטמאו את...,test,sectarian_texts
1974,וכל אשר פרצו את גבול התורה מבאי הברית בהופע כב...,test,sectarian_texts
1975,לצאת ולבוא על פי התורה וישמעו לקול מורה ויתודו...,test,sectarian_texts
1976,ולא ירימו יד על חקי קדשו ומשפטי צדקו ועדוות אמ...,test,sectarian_texts


In [14]:
from BertClassifier import Dataset, train
all_data = df["text"].to_numpy()
all_labels = df["label"].to_numpy()
train_size = int(0.7*len(all_data))
val_size = int(0.1*len(all_data))
test_size = len(all_data) - train_size - val_size
idx = np.arange(len(all_data))
np.random.shuffle(idx)
train_data, train_label = np.concatenate(
    (train_data, all_data[ :train_size])
), np.concatenate((train_label, all_labels[:train_size]))
val_data, val_label = np.concatenate(
    (val_data, all_data[train_size : train_size + val_size])
), np.concatenate((val_label, all_labels[train_size : train_size + val_size]))
test_data, test_label = np.concatenate(
    (test_data, all_data[test_size:])
), np.concatenate((test_label, all_labels[test_size:]))

train_dataset = Dataset(train_data, train_label)
val_dataset = Dataset(val_data, val_label)
test_dataset = Dataset(test_data, test_label)

train(model, train_dataset, val_dataset, LR, EPOCHS)

  0%|          | 0/44 [00:00<?, ?it/s]


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <U15

In [21]:
dir(test_dataset)



array(['sectarian_texts', 'sectarian_texts', 'sectarian_texts', ...,
       'sectarian_texts', 'sectarian_texts', 'sectarian_texts'],
      dtype=object)

In [60]:
idx = np.arange(len(all_data))
np.random.shuffle(idx)
test_size = int(len(all_data) * 0.15)
test_data, test_label = np.concatenate(
    (test_data, all_data[:test_size])
), np.concatenate((test_label, all_labels[:test_size]))
val_data, val_label = np.concatenate(
    (val_data, all_data[test_size : 2 * test_size])
), np.concatenate((val_label, all_labels[test_size : 2 * test_size]))
train_data, train_label = np.concatenate(
    (train_data, all_data[2 * test_size :])
), np.concatenate((train_label, all_labels[2 * test_size :]))

print(size)
train_dataset = Dataset(train_data, train_label)
val_dataset = Dataset(val_data, val_label)
test_dataset = Dataset(test_data, test_label)

train(model, train_dataset, val_dataset, LR, EPOCHS)

,text,training,label
0,שבועות. ובארבעה לשבוע החמישי שמחו וידע אדם שני...,train,0
1,תחת הבל כיא הרגו קין. בשבוע הששי הוליד את אזור...,train,0
2,ובשנת אחת לשבוע הריאשון ליובל החמישי נבנו הבתי...,train,0
3,עוד תשעה בנים ובשבוע החמישי. לקח אנוש את אחותו...,train,0
4,לקח לו קינן אשה את אחותו מהללת לאשה. ותלד לו ב...,train,0
...,...,...,...
1973,מעיר הקדש וישענו על אל בקץ מעל ישראל ויטמאו את...,test,1
1974,וכל אשר פרצו את גבול התורה מבאי הברית בהופע כב...,test,1
1975,לצאת ולבוא על פי התורה וישמעו לקול מורה ויתודו...,test,1
1976,ולא ירימו יד על חקי קדשו ומשפטי צדקו ועדוות אמ...,test,1


In [27]:


from src.parsers.text_reader import read_text

from src.parsers.MorphParser import MorphParser

text_file = f"{BASE_DIR}/data/texts/abegg/dss_nonbib.txt"
yaml_dir = f"{BASE_DIR}/data/yamls"
morph_parser = MorphParser(yaml_dir=yaml_dir)

data, lines = read_text(text_file)


res = []
for i in data:
    res.append({"book":i["book_name"], "scroll":i["scroll_name"]})
df = pd.DataFrame(res)
df
#     
# 
# logger.info("processed the following books}")
# for book in books_list:
#     print(book, end=",")
#     for entry in filtered_data[book]:
#         entry["parsed_morph"] = morph_parser.parse_morph(entry["morph"])
# return filtered_data


[04-05 11:07:55] {/Users/yonatanlou/dev/QumranNLP/src/parsers/text_reader.py:60} INFO - processed 10000 lines
[04-05 11:07:55] {/Users/yonatanlou/dev/QumranNLP/src/parsers/text_reader.py:60} INFO - processed 20000 lines
[04-05 11:07:56] {/Users/yonatanlou/dev/QumranNLP/src/parsers/text_reader.py:60} INFO - processed 30000 lines
[04-05 11:07:56] {/Users/yonatanlou/dev/QumranNLP/src/parsers/text_reader.py:60} INFO - processed 40000 lines
[04-05 11:07:56] {/Users/yonatanlou/dev/QumranNLP/src/parsers/text_reader.py:60} INFO - processed 50000 lines
[04-05 11:07:57] {/Users/yonatanlou/dev/QumranNLP/src/parsers/text_reader.py:60} INFO - processed 60000 lines
[04-05 11:07:57] {/Users/yonatanlou/dev/QumranNLP/src/parsers/text_reader.py:60} INFO - processed 70000 lines
[04-05 11:07:57] {/Users/yonatanlou/dev/QumranNLP/src/parsers/text_reader.py:60} INFO - processed 80000 lines
[04-05 11:07:57] {/Users/yonatanlou/dev/QumranNLP/src/parsers/text_reader.py:60} INFO - processed 90000 lines
[04-05 11:

,book,scroll
0,,CD
1,,CD
2,,CD
3,,CD
4,,CD
...,...,...
342977,,XQ8
342978,,XQ8
342979,,XQ8
342980,,XQ8


In [31]:
df.drop_duplicates()["book"].nunique()

1